In [12]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-30 13:21:50--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-04-30 13:21:50 (9.57 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [13]:
with open('input.txt','r',encoding='utf-8') as f:
    text=f.read()
text

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [14]:
print("length of dataset in characters",len(text))

length of dataset in characters 1115394


In [15]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [16]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [17]:
# create a mapping form character to intergers
stoi ={ch:i for i,ch in enumerate(chars)}
itos= {i:ch for i,ch in enumerate(chars)}

encode = lambda s :[stoi[c] for c in s]
decode = lambda l :''.join(itos[i] for i in l)


print(encode("hii there"))
print(decode(encode("hii there")))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


### alternate tokenizer - tiktoken(openai)

In [22]:
import tiktoken
enc=tiktoken.get_encoding('gpt2')
enc.n_vocab

50257

In [23]:
enc.encode("hii there")


[71, 4178, 612]

### tokenization, train/val split

In [24]:
import torch 
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.type)
print(data[:1000])

torch.Size([1115394]) <built-in method type of Tensor object at 0x12b2240b0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15,

In [25]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

### data loader 

In [27]:
# we split the entire data in chunks or context length 
block_size=8 
train_data[:block_size+1]
# first example input = 18 label = 47
# second example inptu =[18,47] label = 56 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [30]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
print(x,'x')
print(y,'y')

tensor([18, 47, 56, 57, 58,  1, 15, 47]) x
tensor([47, 56, 57, 58,  1, 15, 47, 58]) y


In [33]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
# print(x,'x')
# print(y,'y')
for t in range(block_size):
    context= x[:t+1]
    target=y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [34]:
# we are including antoher dimension btachsize 
torch.manual_seed(1337)
batch_size=4 #how many independent sequences willwe process in parallel 
block_size=8 #what is maximum context length for prediction 

def get_batch(split):
    # generate a small batch of data of inputs x and target y 
    data=train_data if split =='train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack ([data[i+1:i+block_size+1] for i in ix ])
    return x,y 

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)

print('targets:')
print(yb.shape)
print(yb)


print('------')

for b in range(batch_size):
    for t in range(block_size):
        context=xb[b,:t+1]
        target=yb[b,t]
        print(f"when input is {context.tolist()} the target:{target} ")


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------
when input is [24] the target:43 
when input is [24, 43] the target:58 
when input is [24, 43, 58] the target:5 
when input is [24, 43, 58, 5] the target:57 
when input is [24, 43, 58, 5, 57] the target:1 
when input is [24, 43, 58, 5, 57, 1] the target:46 
when input is [24, 43, 58, 5, 57, 1, 46] the target:43 
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target:39 
when input is [44] the target:53 
when input is [44, 53] the target:56 
when input is [44, 53, 56] the target:1 
when input is [44, 53, 56, 1] the target:58 
when input is [44, 53, 56, 1, 58] the target:46 
when input is [44, 

### Simplest baseline: bigram language model, loss generation 

In [40]:
import torch 
import torch.nn as nn 
from torch.nn  import functional as F 
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) :
        super().__init__()
        # each token directly read off the logits for the next token from a lookup table 
        self.token_embedding_table= nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,idx,targets):
        # idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) #(B,T,C)
        
        loss = F.cross_entropy(logits,targets)
        
        return logits,loss
    
    
m=BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)

RuntimeError: Expected target size [4, 65], got [4, 8]

In [43]:
# pytorch cross entropy expects (B,C,T) instead of (B,T,C)
import torch 
import torch.nn as nn 
from torch.nn  import functional as F 
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) :
        super().__init__()
        # each token directly read off the logits for the next token from a lookup table 
        self.token_embedding_table= nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,idx,targets):
        # idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) #(B,T,C)
        
        B,T,C = logits.shape 
        # basically when strecthing(B,T) them into fully connected preserving  the channel(C) 
        logits= logits.view(B*T,C)
        targets = targets.view(B*T)
        
        loss = F.cross_entropy(logits,targets)
        
        return logits,loss
    
    
m=BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [48]:
# now we have added generate funciton i

# pytorch cross entropy expects (B,C,T) instead of (B,T,C)
import torch 
import torch.nn as nn 
from torch.nn  import functional as F 
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) :
        super().__init__()
        # each token directly read off the logits for the next token from a lookup table 
        self.token_embedding_table= nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,idx,targets=None):
        # idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) #(B,T,C)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape 
            # basically when strecthing(B,T) them into fully connected preserving  the channel(C) 
            logits= logits.view(B*T,C)
            targets = targets.view(B*T)
        
            loss = F.cross_entropy(logits,targets)
        
        return logits,loss
    
    def generate(self,idx,max_new_tokens):
        # idx is (B,T ) array of indices in the current context 
        for _ in range(max_new_tokens):
            # get the predictions 
            logits,loss =self(idx)
            # focus only on the last time step 
            logits = logits[:,-1,:] # becomes (B,c)
            # apply softmax to get probablities 
            probs = F.softmax(logits,dim=-1)
            # sample from the distribution 
            idx_next= torch.multinomial(probs,num_samples=1) 
            # appned sampled index to running sequence 
            idx = torch.cat((idx,idx_next),dim=1)
        return idx 
            
    
    
m=BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1),dtype=torch.long)
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


## Train bigram model 

In [49]:
# create torch optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [51]:
batch_size=32 
for steps in range(10000):
    xb,yb = get_batch('train')
    
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward() 
    optimizer.step() 
    
    print(loss.item())

3.7637593746185303
3.760815382003784
3.7295024394989014
3.76991868019104
3.651653528213501
3.7412242889404297
3.5917458534240723
3.801196336746216
3.72733736038208
3.71734881401062
3.74552845954895
3.632333517074585
3.7212233543395996
3.812814950942993
3.794088363647461
3.677765130996704
3.6838455200195312
3.707848310470581
3.7250287532806396
3.6925432682037354
3.6531364917755127
3.659817934036255
3.6261472702026367
3.660931348800659
3.7745466232299805
3.81119704246521
3.8004631996154785
3.7950568199157715
3.7313952445983887
3.705239772796631
3.7860496044158936
3.6133577823638916
3.7759718894958496
3.7864086627960205
3.733623504638672
3.644171953201294
3.681720495223999
3.7327325344085693
3.6357054710388184
3.726031541824341
3.7323036193847656
3.7361340522766113
3.7386937141418457
3.7054803371429443
3.716219663619995
3.7219111919403076
3.6327240467071533
3.833463668823242
3.7487125396728516
3.59930419921875
3.737290143966675
3.830488443374634
3.5564372539520264
3.765251636505127
3.7173

In [52]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))


I weefapsshe d.
S dorithe bld avo the f pir inithe sengr benklos h O:


Foveplod? l lll tt brackinds


In [53]:
import torch 
print(torch.randint(5))

TypeError: randint() received an invalid combination of arguments - got (int), but expected one of:
 * (int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [1]:
i

NameError: name 'i' is not defined